# 0 导入必要的包

In [1]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import ucTools
import funclib
sys.path.append("../../")
import train
sys.path.append("../../")
import test
import time
import joblib
%load_ext autoreload
%autoreload 2

# 1. 读取数据

In [123]:
# 1515 数据
sprot1515 = pd.read_csv('./data/ecoli1515/1515testset_with_unirep.tsv', sep='\t', index_col=0)

In [124]:
colname =  [['id', 'ec_number', 'seq', 'isenzyme', 'isMultiFunctional', 'blattner_id', 'blattner_ec_number' ] +['f'+str(i) for i in range(1,1901)]]
sprot1515.columns =np.array(colname).flatten()
sprot1515.drop_duplicates(subset=['seq'], keep='first', inplace =True)

In [125]:
funclib.table2fasta(sprot1515, './data/sprot1515.fasta')

Write finished


## 2. 序列比对

##   -------------

In [118]:
sprot1515

,id,ec_number,seq,isemzyme,isMultiFunctional,blattner_id,blattner_ec_number,f1,f2,f3,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,P42641,3.6.5.-,MKFVDEASILVVAGDGGNGCVSFRREKYIPKGGPDGGDGGDGGDVW...,True,False,b3183,NaN,0.003056,-0.087184,0.043387,...,0.000332,-0.065542,0.117107,-0.236777,-0.622398,0.012113,0.010506,0.058984,0.061533,0.968014
1,P0AE22,3.1.3.2,MRKITQAISAVCLLFALNSSAVALASSPSPLNPGTNVARLAEQAPI...,True,False,b4055,3.1.3.2,0.001026,0.618393,0.409527,...,0.030307,-0.244636,-0.038585,0.151263,-0.020389,0.095321,-0.031043,-0.881730,-0.196211,0.054455
2,P21499,3.1.13.1,MSQDPFQEREAEKYANPIPSREFILEHLTKREKPASRDELAVELHI...,True,False,b4179,3.1.-.-,0.073632,-0.003258,0.008865,...,0.002123,0.060351,-0.008826,0.006141,-0.129352,-0.075010,0.023712,-0.002246,0.893310,0.140025
3,P39286,3.6.1.-,MSKNKLSKGQQRRVNANHQRRLKTSKEKPDYDDNLFGEPDEGIVIS...,True,False,b4161,3.6.1.-,0.000775,-0.008305,0.041130,...,0.066885,0.020991,0.073577,-0.034962,-0.036755,0.053803,-0.017355,0.100456,-0.202344,0.351100
4,P56100,7.1.1.7,MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI,True,False,b4515,1.10.3.10,0.001811,-0.050181,0.037575,...,0.042016,-0.028950,0.305668,-0.105587,-0.113769,0.680413,0.138797,-0.268528,-0.076093,-0.038212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,P36837,NaN,MNTTTPMGMLQQPRPFFMIFFVELWERFGYYGVQGVLAVFFVKQLG...,False,False,b3496,NaN,0.000070,-0.021424,0.203301,...,0.005911,-0.016855,0.377301,-0.500727,-0.247861,0.752678,-0.005501,0.396191,-0.215832,-0.062812
4746,P77304,NaN,MSTANQKPTESVSLNAFKQPKAFYLIFSIELWERFGYYGLQGIMAV...,False,False,b1634,NaN,0.008628,-0.065382,0.048911,...,-0.001197,-0.101824,0.013007,-0.304450,-0.295525,-0.124492,0.010846,0.032853,-0.155702,-0.183535
4747,P27248,2.1.2.10,MAQQTPLYEQHTLCGARMVDFHGWMMPLHYGSQIDEHHAVRTDAGM...,True,False,b2905,2.1.2.10,0.006723,0.227349,0.029725,...,-0.124573,-0.032099,-0.170996,0.104425,-0.163041,0.183720,-0.038814,0.036993,-0.044558,0.010447
4748,P0AC88,4.2.1.47,MSKVALITGVTGQDGSYLAEFLLEKGYEVHGIKRRASSFNTERVDH...,True,False,b2053,4.2.1.47,0.000665,-0.029179,0.024183,...,0.145032,0.867074,-0.263008,0.009123,0.036783,-0.073805,0.439632,0.014501,-0.205913,-0.005663


In [126]:
test_1515 = sprot1515[['id', 'ec_number', 'blattner_ec_number']]

In [127]:
res_table_head = ['id', 'sseqid', 'pident', 'length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore']
blast_res = pd.read_csv('../../results/1515/balastres.tsv', sep='\t', names=res_table_head)

In [129]:
uctools =  ucTools.ucTools('172.16.25.20')
cnx_mimic = uctools.db_conn()
sql ='''SELECT id, isenzyme, "isMultiFunctional","functionCounts", ec_number, ec_specific_level from tb_sprot where id in ('{0}') '''.format('\',\''.join(blast_res.sseqid.values))
blast_tags = pd.read_sql_query(sql,cnx_mimic)
blast_final = blast_res.merge(blast_tags, left_on='sseqid', right_on='id', how='left')

blast_final.drop_duplicates(subset=['id_x'], keep='first', inplace =True)
blast_final=blast_final[['id_x', 'pident', 'isenzyme', 'isMultiFunctional', 'functionCounts', 'ec_number']]
blast_final=blast_final.rename(columns={'id_x':'id', 'isenzyme':'isenzyme_blast', 'isMultiFunctional':'isMultiFunctional_blast', 'functionCounts':'functionCounts_blast', 'ec_number':'ec_number_blast'})
blast_final.reset_index(drop=True, inplace=True)

In [130]:
blast_final

,id,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast
0,P42641,100.0,True,False,1,3.6.5.-
1,P0AE22,100.0,True,False,1,3.1.3.2
2,P21499,100.0,True,False,1,3.1.13.1
3,P39286,100.0,True,False,1,3.6.1.-
4,P56100,100.0,True,False,1,7.1.1.7
...,...,...,...,...,...,...
4317,P36837,100.0,False,False,1,-
4318,P77304,100.0,False,False,1,-
4319,P27248,100.0,True,False,1,2.1.2.10
4320,P0AC88,100.0,True,False,1,4.2.1.47


In [135]:
test_and_blast = test_1515.merge(blast_final, on='id', how='left')

In [136]:
test_and_blast

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7
...,...,...,...,...,...,...,...,...
4362,P36837,NaN,NaN,100.0,False,False,1.0,-
4363,P77304,NaN,NaN,100.0,False,False,1.0,-
4364,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10
4365,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47


## 3. slice酶非酶预测

In [137]:
def get_isEnzymeRes(querydata, model_file):
    """[获取「酶｜非酶」预测结果]
    Args:
        querydata ([DataFrame]): [需要预测的数据]
        model_file ([string]): [模型文件]
    Returns:
        [DataFrame]: [预测结果、预测概率]
    """
    model = joblib.load(model_file)
    predict = model.predict(querydata)
    predictprob = model.predict_proba(querydata)
    return predict, predictprob[:,1]

In [140]:
sprot1515.iloc[:,7:1909]

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,0.003056,-0.087184,0.043387,-0.018850,-0.623890,0.034710,0.133006,-0.056650,-0.004179,-0.002791,...,0.000332,-0.065542,0.117107,-0.236777,-0.622398,0.012113,0.010506,0.058984,0.061533,0.968014
1,0.001026,0.618393,0.409527,-0.002331,-0.670928,0.036821,-0.148798,-0.038413,-0.002616,0.026154,...,0.030307,-0.244636,-0.038585,0.151263,-0.020389,0.095321,-0.031043,-0.881730,-0.196211,0.054455
2,0.073632,-0.003258,0.008865,-0.012720,-0.072697,-0.026430,-0.033917,-0.000300,-0.001022,0.041117,...,0.002123,0.060351,-0.008826,0.006141,-0.129352,-0.075010,0.023712,-0.002246,0.893310,0.140025
3,0.000775,-0.008305,0.041130,-0.001737,-0.156966,0.009515,0.400612,-0.021405,-0.002408,0.028499,...,0.066885,0.020991,0.073577,-0.034962,-0.036755,0.053803,-0.017355,0.100456,-0.202344,0.351100
4,0.001811,-0.050181,0.037575,-0.005655,0.025414,0.003502,-0.096007,-0.006303,-0.000473,0.012226,...,0.042016,-0.028950,0.305668,-0.105587,-0.113769,0.680413,0.138797,-0.268528,-0.076093,-0.038212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,0.000070,-0.021424,0.203301,0.025054,0.425925,-0.013487,-0.131236,-0.000778,-0.000883,-0.099993,...,0.005911,-0.016855,0.377301,-0.500727,-0.247861,0.752678,-0.005501,0.396191,-0.215832,-0.062812
4746,0.008628,-0.065382,0.048911,-0.215166,-0.197027,0.001949,0.045984,-0.001380,-0.004231,0.022663,...,-0.001197,-0.101824,0.013007,-0.304450,-0.295525,-0.124492,0.010846,0.032853,-0.155702,-0.183535
4747,0.006723,0.227349,0.029725,0.032317,0.064065,0.092823,-0.044260,-0.024524,-0.001700,0.055556,...,-0.124573,-0.032099,-0.170996,0.104425,-0.163041,0.183720,-0.038814,0.036993,-0.044558,0.010447
4748,0.000665,-0.029179,0.024183,-0.005963,-0.175254,0.042627,0.554610,-0.020871,-0.008380,0.105102,...,0.145032,0.867074,-0.263008,0.009123,0.036783,-0.073805,0.439632,0.014501,-0.205913,-0.005663


In [142]:
isEnzyme_model = './model/isenzyme.model'
isEnzyme_pred, isEnzyme_pred_prob = get_isEnzymeRes(querydata=sprot1515.iloc[:,8:1909], model_file=isEnzyme_model )

In [143]:
isEnzyme_pred = pd.DataFrame(isEnzyme_pred)
isEnzyme_pred['isenzyme_slice_prob'] = isEnzyme_pred_prob
isEnzyme_pred['id']=sprot1515.id
isEnzyme_pred.rename(columns={0:'isenzyme_slice_pred'}, inplace=True)

In [146]:
test_and_blast_isenzyme_slice = test_and_blast.merge(isEnzyme_pred, how='left', on='id')

In [148]:
test_and_blast_isenzyme_slice

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1.0,0.988359
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1.0,0.586609
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,1.0,0.659836
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1.0,0.940199
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,0.0,0.088895
...,...,...,...,...,...,...,...,...,...,...
4362,P36837,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN
4363,P77304,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN
4364,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,NaN,NaN
4365,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47,NaN,NaN


In [158]:
def sort_results(result_slice):
    """
    将slice的实验结果排序，并按照推荐顺序以两个矩阵的形式返回
    @pred_top：预测结果排序
    @pred_pb_top：预测结果评分排序
    """
    pred_top =[]
    pred_pb_top =[]
    aac =[]
    for index, row in result_slice.iterrows():
        row_trans= [*row.apply(lambda x: x.split(':')).values]
        row_trans = pd.DataFrame(row_trans).sort_values(by=[1], ascending=False)
        pred_top += [list(np.array(row_trans[0]).astype('int'))]
        pred_pb_top += [list(np.array(row_trans[1]).astype('float'))]
    pred_top = pd.DataFrame(pred_top)
    pred_pb_top = pd.DataFrame(pred_pb_top)
    return pred_top,  pred_pb_top

In [168]:
result_slice = pd.read_csv('../../results/1515/slice_results.txt',  header = None, skiprows=1 ,sep=' ')
slice_pred_top, slice_pred_top_pb = sort_results(result_slice)

In [169]:
dict_label_ec= np.load('../../data/dict_label_ec.npy', allow_pickle=True).item()
slice_pred_ec_top=pd.DataFrame()
for i in range(slice_pred_top.shape[1]):
    slice_pred_ec_top[i]= slice_pred_top[i].apply(lambda x: dict_label_ec.get(x))
    
test_and_blast_isenzyme_slice_pred = pd.concat([test_and_blast_isenzyme_slice, slice_pred_ec_top], axis=1)

In [173]:
test_and_blast_isenzyme_slice_pred

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,...,10,11,12,13,14,15,16,17,18,19
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1.0,0.988359,...,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1.0,0.586609,...,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,1.0,0.659836,...,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1.0,0.940199,...,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,0.0,0.088895,...,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4362,P36837,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,...,2.7.8.23,1.13.11.15,2.4.1.232,4.2.1.140,5.5.1.20,3.2.1.75,1.1.3.42,2.7.1.185,1.13.11.49,1.7.2.8
4363,P77304,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,...,5.1.99.7,6.3.1.20,1.14.11.32,3.6.1.29,1.14.20.6,5.4.99.43,4.5.1.1,2.7.3.3,1.14.16.1,3.1.4.43
4364,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,NaN,NaN,...,3.4.11.24,3.4.22.15,3.4.22.43,3.2.1.177,3.4.21.100,3.2.1.64,3.2.1.11,4.2.2.8,2.4.1.7,4.2.2.1
4365,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47,NaN,NaN,...,2.4.1.62,4.2.99.21,5.4.99.43,3.5.1.90,6.3.2.33,4.2.1.31,3.6.1.62,5.4.4.2,5.4.99.23,3.6.1.45


In [174]:
pd.set_option('display.max_columns', None)

## 4. 添加Ecpred 与DeepEC结果

In [175]:
# 读取预测结果
deepec_results = pd.read_csv('./data/ecoli1515/deepec/DeepEC_Result.txt', sep='\t',names=['id', 'ec_number_deepec'], header=0) #读入文件
deepec_results.ec_number_deepec=deepec_results.apply(lambda x: x['ec_number_deepec'].replace('EC:',''), axis=1)

In [177]:
test_and_blast_isenzyme_slice_pred_deepec= test_and_blast_isenzyme_slice_pred.merge(deepec_results, on='id', how='left')

In [181]:
ecpred_results = pd.read_csv('../../results/1515/ecpred_results.tsv', sep='\t', header=0) #读入文件
ecpred_results = ecpred_results.drop_duplicates(subset=['Protein ID'], keep='first')
ecpred_results = ecpred_results.rename(columns={'Protein ID':'id', 'EC Number':'ecpred_ec_number'})
ecpred_results =  ecpred_results.iloc[:,0:2]

In [182]:
ecpred_results

,id,ecpred_ec_number
0,P42641,3.6.5.-
1,P0AE22,3.1.3.2
2,P21499,3.-.-.-
3,P39286,3.6.1.-
4,P23874,2.7.-.-
...,...,...
4627,P36837,1.-.-.-
4628,P77304,non Enzyme
4629,P27248,2.1.2.10
4630,P0AC88,4.2.1.-


In [183]:
test_blast_slice_deepec_ecpred = test_and_blast_isenzyme_slice_pred_deepec.merge(ecpred_results, on='id', how='left')

In [184]:
test_blast_slice_deepec_ecpred

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,ec_number_deepec,ecpred_ec_number
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1.0,0.988359,3.6.5.2,1.18.6.1,6.3.3.3,3.6.5.3,1.3.7.7,2.3.1.292,7.3.2.7,6.2.1.46,2.7.1.22,6.3.1.5,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27,NaN,3.6.5.-
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1.0,0.586609,3.1.3.2,2.5.1.31,2.7.1.180,3.5.1.1,2.5.1.92,1.11.1.5,4.99.1.3,4.2.1.66,2.4.2.6,3.4.22.8,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,3.1.3.2,3.1.3.2
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,1.0,0.659836,5.2.1.8,3.1.13.1,3.1.26.12,2.7.7.6,2.1.1.85,2.1.1.359,5.6.2.1,2.7.4.24,2.7.11.26,2.7.4.21,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136,3.1.13.1,3.-.-.-
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1.0,0.940199,2.9.1.3,2.7.4.33,6.3.4.5,2.8.2.20,2.7.1.113,2.7.1.76,2.3.1.239,2.7.1.22,2.3.1.240,2.5.1.27,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23,NaN,3.6.1.-
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,0.0,0.088895,7.1.1.2,7.1.1.7,2.1.1.295,3.1.3.78,7.1.1.9,2.1.1.86,4.2.1.78,2.5.1.21,4.4.1.20,7.2.4.2,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5,1.10.3.10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4433,P36837,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,3.2.1.81,4.1.2.9,3.2.1.31,1.17.2.3,1.21.3.3,1.17.9.1,1.3.3.14,1.1.3.45,2.4.1.352,4.1.2.22,2.7.8.23,1.13.11.15,2.4.1.232,4.2.1.140,5.5.1.20,3.2.1.75,1.1.3.42,2.7.1.185,1.13.11.49,1.7.2.8,NaN,1.-.-.-
4434,P77304,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,2.3.1.275,4.4.1.20,1.14.19.77,1.1.1.418,5.3.3.5,3.2.2.21,2.7.8.37,1.17.4.4,1.14.14.18,2.1.1.259,5.1.99.7,6.3.1.20,1.14.11.32,3.6.1.29,1.14.20.6,5.4.99.43,4.5.1.1,2.7.3.3,1.14.16.1,3.1.4.43,NaN,non Enzyme
4435,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,NaN,NaN,3.4.23.49,3.4.23.48,3.4.21.72,2.3.1.251,3.1.1.77,3.1.1.4,3.1.1.1,3.1.1.32,2.4.2.3,3.2.1.94,3.4.11.24,3.4.22.15,3.4.22.43,3.2.1.177,3.4.21.100,3.2.1.64,3.2.1.11,4.2.2.8,2.4.1.7,4.2.2.1,2.1.2.10,2.1.2.10
4436,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47,NaN,NaN,4.1.3.27,3.5.1.42,2.4.2.4,1.1.1.338,2.7.7.6,3.6.1.64,3.6.1.22,2.7.1.134,3.6.1.55,2.7.1.159,2.4.1.62,4.2.99.21,5.4.99.43,3.5.1.90,6.3.2.33,4.2.1.31,3.6.1.62,5.4.4.2,5.4.99.23,3.6.1.45,4.2.1.47,4.2.1.-


In [67]:
def get_ec_rep(balstpred, slicepred, isenzyme, isenzymeprob):
    
    #非酶
    if (isenzyme=='False') and (isenzymeprob ==1):
        return ['', '', '', '','' ,1.0]
    
    #非酶但可能是酶
    if (isenzyme=='False') and (isenzymeprob >=0.5):
        return [slicepred.values[0], slicepred.values[1], slicepred.values[2], slicepred.values[3], slicepred.values[4] ,0.6]
    
    # 全一致
    if balstpred in slicepred.values:
        return [balstpred, '', '', '','' ,1.0]
    
    # 有比对结果
    if str(balstpred)!='NaN':
        return [balstpred, slicepred.values[0], slicepred.values[1],'' ,'' , 0.95] #0.95去掉
    
    # 无比对结果
    if str(balstpred)=='NaN':
        return [slicepred.values[0], slicepred.values[1], slicepred.values[2], slicepred.values[3], slicepred.values[4] , 0.7]
    

In [70]:
aa['ec_pred_final_top1'] = apredslice[0]
aa['ec_pred_final_top2'] = apredslice[1]
aa['ec_pred_final_top3'] = apredslice[2]
aa['ec_pred_final_top4'] = apredslice[3]
aa['ec_pred_final_top5'] = apredslice[4]
aa['ec_pred_final_top1_prob'] = apredslice[5]

In [192]:
deepec = res[(res.ec_number== res.ec_number_deepec) |((res.ec_number.isnull()) & (res.ec_number_deepec.isnull()))]

In [194]:
deepec[(deepec.ec_number.isnull())]

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,ec_number_deepec,ecpred_ec_number
7,P37019,NaN,NaN,100.0,False,False,1.0,-,0.0,0.006490,2.3.1.275,2.7.8.33,2.7.1.202,2.7.1.192,7.6.2.9,2.4.2.53,2.7.1.201,2.7.1.195,4.6.1.1,2.5.1.141,7.2.2.10,7.1.1.7,2.7.8.11,5.3.99.7,2.7.7.41,2.7.8.2,1.12.98.3,7.1.1.1,2.4.1.336,7.2.2.8,NaN,non Enzyme
13,P0AGB3,NaN,NaN,100.0,False,False,1.0,-,0.0,0.009022,2.7.7.85,2.3.2.23,2.7.7.41,2.7.8.37,2.1.1.282,2.7.8.44,7.1.2.2,1.1.1.243,3.1.26.8,4.1.2.55,2.4.1.227,1.12.98.1,7.3.2.4,2.4.1.16,3.5.5.8,2.1.1.63,2.7.9.2,2.7.14.1,2.7.1.22,4.2.99.21,NaN,non Enzyme
14,P37690,NaN,NaN,100.0,False,False,1.0,-,1.0,0.880858,3.4.24.75,3.4.14.13,1.5.99.13,4.3.1.27,2.3.1.157,2.3.1.203,3.4.21.22,2.7.7.23,3.4.21.7,3.2.1.81,3.4.21.68,2.3.1.210,3.4.21.2,3.4.21.4,3.4.21.109,3.1.1.65,3.4.17.13,1.14.15.15,3.4.22.32,2.7.1.45,NaN,non Enzyme
16,P39396,NaN,NaN,100.0,False,False,1.0,-,0.0,0.298930,7.1.1.9,2.3.1.275,7.1.1.7,2.7.8.1,2.1.1.71,5.3.99.7,7.2.1.1,2.7.8.2,2.7.8.33,1.12.98.3,2.4.1.336,1.1.1.418,2.7.1.202,2.5.1.139,1.17.4.4,2.5.1.74,2.5.1.141,5.4.4.1,2.1.1.86,7.1.1.3,NaN,2.-.-.-
18,P0A9E5,NaN,NaN,100.0,False,False,1.0,-,0.0,0.004879,2.1.1.216,3.6.4.12,3.5.4.25,5.1.3.14,3.1.26.8,6.2.1.22,2.4.1.325,3.1.3.68,4.1.1.111,2.7.7.6,2.5.1.17,3.5.1.88,1.14.14.134,3.1.26.5,1.1.1.53,1.14.14.39,5.3.2.1,2.7.8.12,2.7.8.44,1.14.14.37,NaN,non Enzyme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,P77214,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,3.5.2.9,6.1.1.17,1.8.98.6,2.7.1.219,3.5.1.44,1.12.98.4,1.17.1.5,1.8.98.5,1.8.98.4,2.5.1.56,1.4.1.17,4.1.3.25,2.5.1.57,2.3.1.247,1.8.7.3,2.3.1.263,1.2.7.8,4.1.3.6,1.5.3.1,1.2.7.7,NaN,non Enzyme
4430,P0ABN9,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,4.4.1.26,2.5.1.121,1.14.11.45,1.14.13.128,4.2.3.98,4.2.1.131,3.3.2.8,4.2.1.154,1.14.20.6,4.1.1.44,3.6.2.1,1.8.4.11,3.6.1.22,4.2.1.94,4.1.1.97,5.1.99.8,6.3.1.12,1.16.3.1,1.3.7.12,3.6.1.25,NaN,no Prediction
4433,P36837,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,3.2.1.81,4.1.2.9,3.2.1.31,1.17.2.3,1.21.3.3,1.17.9.1,1.3.3.14,1.1.3.45,2.4.1.352,4.1.2.22,2.7.8.23,1.13.11.15,2.4.1.232,4.2.1.140,5.5.1.20,3.2.1.75,1.1.3.42,2.7.1.185,1.13.11.49,1.7.2.8,NaN,1.-.-.-
4434,P77304,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,2.3.1.275,4.4.1.20,1.14.19.77,1.1.1.418,5.3.3.5,3.2.2.21,2.7.8.37,1.17.4.4,1.14.14.18,2.1.1.259,5.1.99.7,6.3.1.20,1.14.11.32,3.6.1.29,1.14.20.6,5.4.99.43,4.5.1.1,2.7.3.3,1.14.16.1,3.1.4.43,NaN,non Enzyme


In [185]:
res = test_blast_slice_deepec_ecpred

In [186]:
blast =res[(res.ec_number== res.ec_number_blast)|((res.ec_number.isnull()) & (res.ec_number_blast=='-'))]

In [206]:
blast[~(blast.ec_number.isnull())]

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,ec_number_deepec,ecpred_ec_number,ecpp
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1.0,0.988359,3.6.5.2,1.18.6.1,6.3.3.3,3.6.5.3,1.3.7.7,2.3.1.292,7.3.2.7,6.2.1.46,2.7.1.22,6.3.1.5,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27,NaN,3.6.5.-,3.6.5.-
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1.0,0.586609,3.1.3.2,2.5.1.31,2.7.1.180,3.5.1.1,2.5.1.92,1.11.1.5,4.99.1.3,4.2.1.66,2.4.2.6,3.4.22.8,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,3.1.3.2,3.1.3.2,3.1.3.2
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,1.0,0.659836,5.2.1.8,3.1.13.1,3.1.26.12,2.7.7.6,2.1.1.85,2.1.1.359,5.6.2.1,2.7.4.24,2.7.11.26,2.7.4.21,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136,3.1.13.1,3.-.-.-,3.1.13.1
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1.0,0.940199,2.9.1.3,2.7.4.33,6.3.4.5,2.8.2.20,2.7.1.113,2.7.1.76,2.3.1.239,2.7.1.22,2.3.1.240,2.5.1.27,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23,NaN,3.6.1.-,3.6.1.-
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,0.0,0.088895,7.1.1.2,7.1.1.7,2.1.1.295,3.1.3.78,7.1.1.9,2.1.1.86,4.2.1.78,2.5.1.21,4.4.1.20,7.2.4.2,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5,1.10.3.10,NaN,7.1.1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4428,P16676,7.3.2.3,3.6.3.25,100.0,True,False,1.0,7.3.2.3,NaN,NaN,1.14.14.155,1.1.1.193,4.1.1.95,3.7.1.12,6.3.2.51,1.2.1.87,3.2.2.28,3.5.1.100,4.1.3.32,6.3.2.47,7.4.2.11,4.3.1.25,2.7.4.26,4.1.3.25,2.7.1.185,1.2.7.10,4.2.99.21,1.14.11.28,3.1.3.91,3.5.4.26,3.6.3.25,3.6.3.25,7.3.2.3
4431,P76330,2.7.7.65,2.7.7.65,100.0,True,False,1.0,2.7.7.65,NaN,NaN,2.7.7.65,1.8.98.6,1.3.1.2,1.5.99.15,1.8.98.5,1.12.98.4,1.2.7.8,1.17.99.6,7.1.1.8,1.2.7.10,1.3.5.4,1.97.1.12,1.8.98.4,1.3.5.1,7.2.2.7,1.2.7.7,1.21.98.4,7.1.1.6,2.8.4.3,1.3.1.1,2.7.7.65,2.7.7.-,2.7.7.65
4432,P36655,1.8.1.8,1.8.1.8,100.0,True,False,1.0,1.8.1.8,NaN,NaN,2.7.1.201,2.7.1.195,5.3.99.7,2.7.8.5,3.6.1.27,2.7.8.24,2.7.1.198,7.2.4.1,2.5.1.139,2.5.1.145,2.7.8.35,2.5.1.137,2.7.1.174,1.12.98.3,2.7.1.192,2.7.7.74,2.5.1.62,2.7.8.34,1.8.98.1,2.7.8.11,1.8.1.8,1.8.1.8,1.8.1.8
4435,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,NaN,NaN,3.4.23.49,3.4.23.48,3.4.21.72,2.3.1.251,3.1.1.77,3.1.1.4,3.1.1.1,3.1.1.32,2.4.2.3,3.2.1.94,3.4.11.24,3.4.22.15,3.4.22.43,3.2.1.177,3.4.21.100,3.2.1.64,3.2.1.11,4.2.2.8,2.4.1.7,4.2.2.1,2.1.2.10,2.1.2.10,2.1.2.10


In [189]:
ecpred = res[(res.ec_number== res.ecpred_ec_number) |((res.ec_number.isnull()) & (res.ecpred_ec_number=='non Enzyme'))]

In [202]:
ecpred[~(ecpred.ec_number.isnull())]

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,ec_number_deepec,ecpred_ec_number
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1.0,0.988359,3.6.5.2,1.18.6.1,6.3.3.3,3.6.5.3,1.3.7.7,2.3.1.292,7.3.2.7,6.2.1.46,2.7.1.22,6.3.1.5,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27,NaN,3.6.5.-
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1.0,0.586609,3.1.3.2,2.5.1.31,2.7.1.180,3.5.1.1,2.5.1.92,1.11.1.5,4.99.1.3,4.2.1.66,2.4.2.6,3.4.22.8,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,3.1.3.2,3.1.3.2
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1.0,0.940199,2.9.1.3,2.7.4.33,6.3.4.5,2.8.2.20,2.7.1.113,2.7.1.76,2.3.1.239,2.7.1.22,2.3.1.240,2.5.1.27,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23,NaN,3.6.1.-
8,P0DTT0,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1.0,0.604362,3.6.5.3,2.7.7.4,2.3.1.283,2.7.4.8,2.8.1.14,3.6.5.5,2.4.1.174,7.6.2.4,2.4.1.356,3.4.11.4,2.3.1.20,2.7.1.164,7.3.2.7,2.4.1.266,2.3.1.239,2.4.1.38,4.1.1.81,2.3.1.240,2.9.1.2,2.8.1.13,NaN,3.6.5.-
9,Q46865,3.1.-.-,3.1.-.-,100.0,True,False,1.0,3.1.-.-,0.0,0.060774,2.7.8.7,3.5.1.135,4.4.1.26,3.1.13.5,1.14.11.45,4.2.3.12,3.1.21.4,4.2.1.154,5.1.99.8,2.5.1.109,3.6.1.56,5.3.3.4,3.5.1.5,4.2.1.104,3.5.4.35,2.5.1.107,4.1.3.40,1.14.11.33,1.14.11.64,4.2.1.94,NaN,3.1.-.-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,P39409,1.97.1.-,NaN,100.0,True,False,1.0,1.97.1.-,NaN,NaN,6.1.1.14,2.7.7.48,2.3.1.118,6.1.1.16,3.1.11.1,1.8.5.1,3.5.4.16,1.5.5.3,3.7.1.18,6.1.1.2,1.3.8.11,1.13.11.80,3.13.1.3,2.4.2.48,1.8.7.1,1.20.4.2,4.2.3.33,1.1.1.88,6.1.2.2,2.1.1.13,NaN,1.97.1.-
4423,P25718,3.2.1.1,3.2.1.1,100.0,True,False,1.0,3.2.1.1,NaN,NaN,3.1.3.5,3.6.4.13,3.2.1.1,6.3.4.19,3.1.21.5,6.3.3.4,2.3.1.161,7.4.2.4,2.3.1.282,2.8.2.2,7.2.2.1,6.3.3.6,2.3.1.193,2.8.2.1,4.4.1.37,6.3.5.4,6.3.5.1,2.8.2.3,2.4.1.92,7.3.2.7,3.2.1.1,3.2.1.1
4424,P0AA84,2.7.8.-,2.7.8.-,100.0,True,False,1.0,2.7.8.-,NaN,NaN,7.1.2.2,2.3.1.274,3.1.3.64,2.5.1.108,3.1.3.95,2.7.1.21,2.7.1.130,2.7.1.113,2.7.1.19,2.3.1.193,3.1.3.34,3.1.3.5,2.5.1.61,3.1.1.74,2.1.1.160,3.1.1.1,4.1.3.1,2.7.1.173,2.7.1.76,3.1.11.5,NaN,2.7.8.-
4432,P36655,1.8.1.8,1.8.1.8,100.0,True,False,1.0,1.8.1.8,NaN,NaN,2.7.1.201,2.7.1.195,5.3.99.7,2.7.8.5,3.6.1.27,2.7.8.24,2.7.1.198,7.2.4.1,2.5.1.139,2.5.1.145,2.7.8.35,2.5.1.137,2.7.1.174,1.12.98.3,2.7.1.192,2.7.7.74,2.5.1.62,2.7.8.34,1.8.98.1,2.7.8.11,1.8.1.8,1.8.1.8


In [197]:
res['ecpp'] = res.apply(lambda x: x.ec_number_blast if x.ec_number_blast !='-' else x[0], axis=1)

In [198]:
res

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,ec_number_deepec,ecpred_ec_number,ecpp
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1.0,0.988359,3.6.5.2,1.18.6.1,6.3.3.3,3.6.5.3,1.3.7.7,2.3.1.292,7.3.2.7,6.2.1.46,2.7.1.22,6.3.1.5,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27,NaN,3.6.5.-,3.6.5.-
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1.0,0.586609,3.1.3.2,2.5.1.31,2.7.1.180,3.5.1.1,2.5.1.92,1.11.1.5,4.99.1.3,4.2.1.66,2.4.2.6,3.4.22.8,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,3.1.3.2,3.1.3.2,3.1.3.2
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,1.0,0.659836,5.2.1.8,3.1.13.1,3.1.26.12,2.7.7.6,2.1.1.85,2.1.1.359,5.6.2.1,2.7.4.24,2.7.11.26,2.7.4.21,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136,3.1.13.1,3.-.-.-,3.1.13.1
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1.0,0.940199,2.9.1.3,2.7.4.33,6.3.4.5,2.8.2.20,2.7.1.113,2.7.1.76,2.3.1.239,2.7.1.22,2.3.1.240,2.5.1.27,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23,NaN,3.6.1.-,3.6.1.-
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,0.0,0.088895,7.1.1.2,7.1.1.7,2.1.1.295,3.1.3.78,7.1.1.9,2.1.1.86,4.2.1.78,2.5.1.21,4.4.1.20,7.2.4.2,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5,1.10.3.10,NaN,7.1.1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4433,P36837,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,3.2.1.81,4.1.2.9,3.2.1.31,1.17.2.3,1.21.3.3,1.17.9.1,1.3.3.14,1.1.3.45,2.4.1.352,4.1.2.22,2.7.8.23,1.13.11.15,2.4.1.232,4.2.1.140,5.5.1.20,3.2.1.75,1.1.3.42,2.7.1.185,1.13.11.49,1.7.2.8,NaN,1.-.-.-,3.2.1.81
4434,P77304,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,2.3.1.275,4.4.1.20,1.14.19.77,1.1.1.418,5.3.3.5,3.2.2.21,2.7.8.37,1.17.4.4,1.14.14.18,2.1.1.259,5.1.99.7,6.3.1.20,1.14.11.32,3.6.1.29,1.14.20.6,5.4.99.43,4.5.1.1,2.7.3.3,1.14.16.1,3.1.4.43,NaN,non Enzyme,2.3.1.275
4435,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,NaN,NaN,3.4.23.49,3.4.23.48,3.4.21.72,2.3.1.251,3.1.1.77,3.1.1.4,3.1.1.1,3.1.1.32,2.4.2.3,3.2.1.94,3.4.11.24,3.4.22.15,3.4.22.43,3.2.1.177,3.4.21.100,3.2.1.64,3.2.1.11,4.2.2.8,2.4.1.7,4.2.2.1,2.1.2.10,2.1.2.10,2.1.2.10
4436,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47,NaN,NaN,4.1.3.27,3.5.1.42,2.4.2.4,1.1.1.338,2.7.7.6,3.6.1.64,3.6.1.22,2.7.1.134,3.6.1.55,2.7.1.159,2.4.1.62,4.2.99.21,5.4.99.43,3.5.1.90,6.3.2.33,4.2.1.31,3.6.1.62,5.4.4.2,5.4.99.23,3.6.1.45,4.2.1.47,4.2.1.-,4.2.1.47


In [222]:
blast =res[(res.ec_number== res.ecpp)]

In [208]:
blast

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,ec_number_deepec,ecpred_ec_number,ecpp
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1.0,0.988359,3.6.5.2,1.18.6.1,6.3.3.3,3.6.5.3,1.3.7.7,2.3.1.292,7.3.2.7,6.2.1.46,2.7.1.22,6.3.1.5,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27,NaN,3.6.5.-,3.6.5.-
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1.0,0.586609,3.1.3.2,2.5.1.31,2.7.1.180,3.5.1.1,2.5.1.92,1.11.1.5,4.99.1.3,4.2.1.66,2.4.2.6,3.4.22.8,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,3.1.3.2,3.1.3.2,3.1.3.2
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,1.0,0.659836,5.2.1.8,3.1.13.1,3.1.26.12,2.7.7.6,2.1.1.85,2.1.1.359,5.6.2.1,2.7.4.24,2.7.11.26,2.7.4.21,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136,3.1.13.1,3.-.-.-,3.1.13.1
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1.0,0.940199,2.9.1.3,2.7.4.33,6.3.4.5,2.8.2.20,2.7.1.113,2.7.1.76,2.3.1.239,2.7.1.22,2.3.1.240,2.5.1.27,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23,NaN,3.6.1.-,3.6.1.-
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,0.0,0.088895,7.1.1.2,7.1.1.7,2.1.1.295,3.1.3.78,7.1.1.9,2.1.1.86,4.2.1.78,2.5.1.21,4.4.1.20,7.2.4.2,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5,1.10.3.10,NaN,7.1.1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4428,P16676,7.3.2.3,3.6.3.25,100.0,True,False,1.0,7.3.2.3,NaN,NaN,1.14.14.155,1.1.1.193,4.1.1.95,3.7.1.12,6.3.2.51,1.2.1.87,3.2.2.28,3.5.1.100,4.1.3.32,6.3.2.47,7.4.2.11,4.3.1.25,2.7.4.26,4.1.3.25,2.7.1.185,1.2.7.10,4.2.99.21,1.14.11.28,3.1.3.91,3.5.4.26,3.6.3.25,3.6.3.25,7.3.2.3
4431,P76330,2.7.7.65,2.7.7.65,100.0,True,False,1.0,2.7.7.65,NaN,NaN,2.7.7.65,1.8.98.6,1.3.1.2,1.5.99.15,1.8.98.5,1.12.98.4,1.2.7.8,1.17.99.6,7.1.1.8,1.2.7.10,1.3.5.4,1.97.1.12,1.8.98.4,1.3.5.1,7.2.2.7,1.2.7.7,1.21.98.4,7.1.1.6,2.8.4.3,1.3.1.1,2.7.7.65,2.7.7.-,2.7.7.65
4432,P36655,1.8.1.8,1.8.1.8,100.0,True,False,1.0,1.8.1.8,NaN,NaN,2.7.1.201,2.7.1.195,5.3.99.7,2.7.8.5,3.6.1.27,2.7.8.24,2.7.1.198,7.2.4.1,2.5.1.139,2.5.1.145,2.7.8.35,2.5.1.137,2.7.1.174,1.12.98.3,2.7.1.192,2.7.7.74,2.5.1.62,2.7.8.34,1.8.98.1,2.7.8.11,1.8.1.8,1.8.1.8,1.8.1.8
4435,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,NaN,NaN,3.4.23.49,3.4.23.48,3.4.21.72,2.3.1.251,3.1.1.77,3.1.1.4,3.1.1.1,3.1.1.32,2.4.2.3,3.2.1.94,3.4.11.24,3.4.22.15,3.4.22.43,3.2.1.177,3.4.21.100,3.2.1.64,3.2.1.11,4.2.2.8,2.4.1.7,4.2.2.1,2.1.2.10,2.1.2.10,2.1.2.10


In [223]:
res[(res.ec_number_blast=='-') & (res.ec_number.isnull())][['ec_number', 'ecpp']]

,ec_number,ecpp
7,NaN,2.3.1.275
13,NaN,2.7.7.85
14,NaN,3.4.24.75
16,NaN,7.1.1.9
18,NaN,2.1.1.216
...,...,...
4429,NaN,3.5.2.9
4430,NaN,4.4.1.26
4433,NaN,3.2.1.81
4434,NaN,2.3.1.275


In [261]:

pd.options.display.max_rows = 20

In [262]:
res[(~res.ec_number.isnull()) & (res.ec_number != res.ec_number_blast)]

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,ec_number_deepec,ecpred_ec_number,ecpp,isenzyme_pred
6,P23837,2.7.13.3; 3.1.3.-,2.7.13.3;3.1.3.-,100.0,True,True,2.0,2.7.13.3,1.0,0.953441,2.7.13.3,2.4.1.347,2.3.1.85,2.7.8.46,2.4.1.331,1.3.1.39,1.14.14.19,1.14.14.32,2.4.1.277,3.1.2.14,2.3.1.268,2.4.1.316,2.4.1.311,1.10.3.1,1.8.3.1,1.1.1.243,1.14.14.149,2.4.1.276,1.14.13.233,2.4.1.322,2.7.13.3,2.7.13.3,2.7.13.3,True
11,P0AG24,2.7.6.5; 3.1.7.2,2.7.6.5;3.1.7.2,100.0,True,True,2.0,3.1.7.2,1.0,0.615924,3.1.7.2,2.7.6.5,2.7.7.6,6.1.1.14,6.3.2.20,4.3.1.1,4.99.1.12,2.4.1.1,3.5.1.114,6.2.1.22,4.6.1.22,6.3.2.51,4.2.1.2,3.5.1.60,1.14.13.43,5.4.3.6,2.1.1.13,2.1.1.239,1.7.7.1,4.1.1.18,3.1.7.2,3.1.-.-,3.1.7.2,True
41,P0AGC5,4.2.2.n1,4.2.2.n1,100.0,True,False,1.0,4.2.2.-,1.0,0.983417,4.2.2.3,3.5.1.97,5.1.3.17,3.5.2.6,3.1.31.1,3.2.1.96,1.1.5.5,5.4.99.38,3.5.1.28,3.4.24.55,2.4.1.232,2.4.1.184,2.7.1.180,1.17.2.2,2.4.2.36,3.5.1.60,1.8.5.3,3.5.1.79,3.1.1.49,3.5.1.12,4.2.2.n1,4.2.2.n1,4.2.2.-,True
94,P0A840,3.1.3.5; 3.1.3.6; 3.6.1.11,3.1.3.5;3.1.3.6;3.6.1.11,100.0,True,True,3.0,3.1.3.5,1.0,0.965913,3.6.1.11,3.1.3.6,3.1.3.5,2.7.2.4,4.99.1.3,1.21.4.4,3.1.3.63,1.17.7.4,2.5.1.76,2.2.1.7,3.2.2.1,1.21.4.3,2.5.1.65,1.1.1.409,4.1.1.21,1.1.1.399,1.21.4.2,2.4.2.6,3.3.2.1,2.7.4.26,3.6.1.11,3.1.3.5,3.1.3.5,True
95,P0A840,3.1.3.5; 3.1.3.6; 3.6.1.11,3.1.3.5;3.1.3.6;3.6.1.11,100.0,True,True,3.0,3.1.3.5,1.0,0.965913,3.6.1.11,3.1.3.6,3.1.3.5,2.7.2.4,4.99.1.3,1.21.4.4,3.1.3.63,1.17.7.4,2.5.1.76,2.2.1.7,3.2.2.1,1.21.4.3,2.5.1.65,1.1.1.409,4.1.1.21,1.1.1.399,1.21.4.2,2.4.2.6,3.3.2.1,2.7.4.26,3.1.3.5,3.1.3.5,3.1.3.5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4273,P0A9M5,2.4.2.-; 2.4.2.22,2.4.2.22,100.0,True,True,2.0,2.4.2.-,NaN,NaN,3.1.11.1,3.6.1.55,2.5.1.18,3.4.17.6,2.3.2.23,3.6.1.67,3.6.1.12,2.1.1.282,2.3.1.259,5.2.1.4,4.2.3.98,2.1.1.86,1.16.1.6,2.5.1.151,6.3.1.20,4.2.1.138,4.2.3.89,1.3.99.17,3.6.1.52,3.3.2.11,2.4.2.22,2.4.2.22,2.4.2.-,True
4274,P09099,2.7.1.17; 2.7.1.-,2.7.1.17,100.0,True,True,2.0,2.7.1.17,NaN,NaN,7.1.1.2,3.6.1.26,2.4.1.43,3.4.23.39,3.4.24.55,2.4.1.258,2.4.99.19,3.4.24.89,3.5.2.6,2.4.1.175,2.4.1.226,1.1.1.94,2.4.1.260,1.7.6.1,2.5.1.2,3.5.1.28,3.2.1.96,2.4.1.274,1.1.99.3,2.4.1.261,2.7.1.17,2.-.-.-,2.7.1.17,True
4385,P45470,3.1.2.-; 3.5.1.-; 3.5.1.124,NaN,100.0,True,True,3.0,3.5.1.124,NaN,NaN,7.1.1.2,7.2.2.4,2.7.1.198,2.7.7.74,2.7.8.34,7.2.2.14,3.2.1.97,2.5.1.150,2.5.1.130,2.3.1.269,1.12.98.3,7.1.1.1,6.3.2.2,3.4.21.89,2.7.8.14,2.4.1.257,2.4.1.132,7.2.2.10,7.2.4.1,3.5.1.41,NaN,3.-.-.-,3.5.1.124,True
4388,Q46845,1.8.4.-; 1.11.1.-,1.11.1.-;1.8.4.-,100.0,True,True,2.0,1.8.4.-,NaN,NaN,2.1.1.199,2.1.1.33,2.7.1.71,3.1.1.1,2.1.1.61,5.3.3.8,2.1.1.181,2.1.1.264,2.1.1.173,5.1.2.3,5.4.2.12,2.5.1.75,4.2.1.17,1.1.1.49,1.18.6.1,2.1.1.80,2.1.1.166,1.6.5.4,2.1.1.8,2.1.1.345,NaN,2.5.1.18,1.8.4.-,True


In [228]:
res['isenzyme_pred'] = res.apply(lambda x:x.isenzyme_slice_pred if x.isenzyme_blast.isnull() else x.isenzyme_blast ,axis=1)

AttributeError: 'bool' object has no attribute 'isnull'

In [226]:
res.to_excel('../../results/1515/mulit_results.xlsx', index=None) 

In [ ]:
pd.read_csv('../../results/1515/ecpred_results.tsv', sep='\t', header=0) #读入文件

In [240]:
modified = pd.read_excel('../../results/1515/mulit_results_manual_modified.xlsx')

In [245]:
aa = modified[(modified.ec_number==modified.ec_pred)]

In [249]:
sprot1515[~sprot1515.isenzyme]

id ec_number                                                seq  \
7     P37019       NaN  MKTDTPSLETPQAARLRRRQLIRQLLERDKTPLAILFMAAVVGTLV...   
13    P0AGB3       NaN  MTDKMQSLALAPVGNLDSYIRAANAWPMLSADEERALAEKLHYHGD...   
14    P37690       NaN  MTRAVKPRRFAIRPIIYASVLSAGVLLCAFSAHADERDQLKSIQAD...   
16    P39396       NaN  MDTKKIFKHIPWVILGIIGAFCLAVVALRRGEHISALWIVVASVSV...   
18    P0A9E5       NaN  MIPEKRIIRRIQSGGCAIHCQDCSISQLCIPFTLNEHELDQLDNII...   
...      ...       ...                                                ...   
4741  P77214       NaN  MKKALQVAMFSLFTVIGFNAQANEHHHETMSEAQPQVISATGVVKG...   
4742  P0ABN9       NaN  MLFTIQLIIILICLFYGARKGGIALGLLGGIGLVILVFVFHLQPGK...   
4745  P36837       NaN  MNTTTPMGMLQQPRPFFMIFFVELWERFGYYGVQGVLAVFFVKQLG...   
4746  P77304       NaN  MSTANQKPTESVSLNAFKQPKAFYLIFSIELWERFGYYGLQGIMAV...   
4749  P0A6N8       NaN  MPRANEIKKGMVLNYNGKLLLVKDIDIQSPTARGAATLYKMRFSDV...   

      isenzyme  isMultiFunctional blattner_id blattner_ec_number        f1  \
7        False              False       b0155                NaN  0.004370   
13       False              False       b3461                NaN  0.000724   
14       False              False       b3613                NaN  0.087595   
16       False              False       b4354                NaN  0.001285   
18       False              False       b1334                NaN  0.009290   
...        ...                ...         ...                ...       ...   
4741     False              False       b0573                NaN  0.001910   
4742     False              False       b4123                NaN  0.003877   
4745     False              False       b3496                NaN  0.000070   
4746     False              False       b1634                NaN  0.008628   
4749     False              False       b2171                NaN  0.000589   

            f2        f3        f4        f5        f6        f7        f8  \
7     0.004021  0.022877 -0.141994 -0.382760  0.077454 -0.013590 -0.008866   
13    0.009471  0.042763 -0.006152  0.332184  0.000323 -0.171502 -0.060499   
14   -0.000547 -0.075345 -0.226387 -0.034486 -0.016566 -0.396511 -0.226322   
16   -0.005200  0.038819 -0.016078 -0.009708  0.126894 -0.016121 -0.014880   
18    0.500250  0.329449 -0.251223 -0.534687  0.004159  0.238510 -0.034866   
...        ...       ...       ...       ...       ...       ...       ...   
4741 -0.074665  0.099424 -0.038839 -0.214466 -0.050259 -0.432040 -0.022034   
4742 -0.032313  0.031977 -0.055821 -0.178217  0.154485 -0.094127 -0.131303   
4745 -0.021424  0.203301  0.025054  0.425925 -0.013487 -0.131236 -0.000778   
4746 -0.065382  0.048911 -0.215166 -0.197027  0.001949  0.045984 -0.001380   
4749  0.032014  0.013608 -0.008443 -0.115612  0.028364 -0.153695 -0.032118   

            f9       f10       f11       f12       f13       f14       f15  \
7    -0.002961  0.678808  0.754905  0.000759  0.032332 -0.100989  0.019440   
13   -0.005948 -0.006750 -0.043102 -0.025274  0.011199  0.100780  0.006136   
14    0.007313  0.046911 -0.070742 -0.106144  0.010924 -0.259933  0.000635   
16   -0.002461 -0.399723 -0.009344 -0.000398  0.041058 -0.146212  0.037882   
18   -0.005264  0.000048 -0.012469 -0.001118  0.017716  0.378760  0.047818   
...        ...       ...       ...       ...       ...       ...       ...   
4741  0.030598  0.151941  0.203657 -0.033349 -0.122936 -0.001793 -0.003111   
4742 -0.015271  0.002664  0.652455 -0.004379  0.059436 -0.182538  0.107167   
4745 -0.000883 -0.099993  0.836958  0.000932  0.031655 -0.030638  0.086026   
4746 -0.004231  0.022663  0.821562 -0.000184  0.113237 -0.018413  0.045726   
4749 -0.000680  0.051763 -0.121068 -0.000767  0.050263  0.110978  0.005803   

           f16       f17       f18       f19       f20       f21       f22  \
7    -0.000033  0.168928 -0.010399  0.124722 -0.002627 -0.012438 -0.397947   
13   -0.000664 -0.020253 -0.208283  0.033201 -0.036375 -0.064554 -0.020972   
14   -0.000298  0.004391 -0.154934  0.02

In [252]:
modified[(modified.ec_number.isnull()) & modified.ec_pred.isnull()]

,id,ec_number,ec_number_deepec,ecpred_ec_number,ec_number_blast,ec_pred,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
7,P37019,NaN,NaN,non Enzyme,-,NaN,2.3.1.275,2.7.8.33,2.7.1.202,2.7.1.192,7.6.2.9,2.4.2.53,2.7.1.201,2.7.1.195,4.6.1.1,2.5.1.141,7.2.2.10,7.1.1.7,2.7.8.11,5.3.99.7,2.7.7.41,2.7.8.2,1.12.98.3,7.1.1.1,2.4.1.336,7.2.2.8
13,P0AGB3,NaN,NaN,non Enzyme,-,NaN,2.7.7.85,2.3.2.23,2.7.7.41,2.7.8.37,2.1.1.282,2.7.8.44,7.1.2.2,1.1.1.243,3.1.26.8,4.1.2.55,2.4.1.227,1.12.98.1,7.3.2.4,2.4.1.16,3.5.5.8,2.1.1.63,2.7.9.2,2.7.14.1,2.7.1.22,4.2.99.21
14,P37690,NaN,NaN,non Enzyme,-,NaN,3.4.24.75,3.4.14.13,1.5.99.13,4.3.1.27,2.3.1.157,2.3.1.203,3.4.21.22,2.7.7.23,3.4.21.7,3.2.1.81,3.4.21.68,2.3.1.210,3.4.21.2,3.4.21.4,3.4.21.109,3.1.1.65,3.4.17.13,1.14.15.15,3.4.22.32,2.7.1.45
16,P39396,NaN,NaN,2.-.-.-,-,NaN,7.1.1.9,2.3.1.275,7.1.1.7,2.7.8.1,2.1.1.71,5.3.99.7,7.2.1.1,2.7.8.2,2.7.8.33,1.12.98.3,2.4.1.336,1.1.1.418,2.7.1.202,2.5.1.139,1.17.4.4,2.5.1.74,2.5.1.141,5.4.4.1,2.1.1.86,7.1.1.3
18,P0A9E5,NaN,NaN,non Enzyme,-,NaN,2.1.1.216,3.6.4.12,3.5.4.25,5.1.3.14,3.1.26.8,6.2.1.22,2.4.1.325,3.1.3.68,4.1.1.111,2.7.7.6,2.5.1.17,3.5.1.88,1.14.14.134,3.1.26.5,1.1.1.53,1.14.14.39,5.3.2.1,2.7.8.12,2.7.8.44,1.14.14.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,P77214,NaN,NaN,non Enzyme,-,NaN,3.5.2.9,6.1.1.17,1.8.98.6,2.7.1.219,3.5.1.44,1.12.98.4,1.17.1.5,1.8.98.5,1.8.98.4,2.5.1.56,1.4.1.17,4.1.3.25,2.5.1.57,2.3.1.247,1.8.7.3,2.3.1.263,1.2.7.8,4.1.3.6,1.5.3.1,1.2.7.7
4430,P0ABN9,NaN,NaN,no Prediction,-,NaN,4.4.1.26,2.5.1.121,1.14.11.45,1.14.13.128,4.2.3.98,4.2.1.131,3.3.2.8,4.2.1.154,1.14.20.6,4.1.1.44,3.6.2.1,1.8.4.11,3.6.1.22,4.2.1.94,4.1.1.97,5.1.99.8,6.3.1.12,1.16.3.1,1.3.7.12,3.6.1.25
4433,P36837,NaN,NaN,1.-.-.-,-,NaN,3.2.1.81,4.1.2.9,3.2.1.31,1.17.2.3,1.21.3.3,1.17.9.1,1.3.3.14,1.1.3.45,2.4.1.352,4.1.2.22,2.7.8.23,1.13.11.15,2.4.1.232,4.2.1.140,5.5.1.20,3.2.1.75,1.1.3.42,2.7.1.185,1.13.11.49,1.7.2.8
4434,P77304,NaN,NaN,non Enzyme,-,NaN,2.3.1.275,4.4.1.20,1.14.19.77,1.1.1.418,5.3.3.5,3.2.2.21,2.7.8.37,1.17.4.4,1.14.14.18,2.1.1.259,5.1.99.7,6.3.1.20,1.14.11.32,3.6.1.29,1.14.20.6,5.4.99.43,4.5.1.1,2.7.3.3,1.14.16.1,3.1.4.43


In [253]:
res

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,ec_number_deepec,ecpred_ec_number,ecpp,isenzyme_pred
0,P42641,3.6.5.-,NaN,100.0,True,False,1.0,3.6.5.-,1.0,0.988359,3.6.5.2,1.18.6.1,6.3.3.3,3.6.5.3,1.3.7.7,2.3.1.292,7.3.2.7,6.2.1.46,2.7.1.22,6.3.1.5,6.2.1.33,6.3.5.4,2.3.1.239,2.3.1.240,6.3.1.15,6.2.1.32,6.2.1.51,2.3.1.111,6.2.1.1,6.2.1.27,NaN,3.6.5.-,3.6.5.-,True
1,P0AE22,3.1.3.2,3.1.3.2,100.0,True,False,1.0,3.1.3.2,1.0,0.586609,3.1.3.2,2.5.1.31,2.7.1.180,3.5.1.1,2.5.1.92,1.11.1.5,4.99.1.3,4.2.1.66,2.4.2.6,3.4.22.8,2.5.1.28,2.4.1.232,4.6.1.13,3.5.1.3,1.13.11.29,3.5.1.28,3.1.16.1,1.17.9.1,3.1.4.41,1.14.99.39,3.1.3.2,3.1.3.2,3.1.3.2,True
2,P21499,3.1.13.1,3.1.-.-,100.0,True,False,1.0,3.1.13.1,1.0,0.659836,5.2.1.8,3.1.13.1,3.1.26.12,2.7.7.6,2.1.1.85,2.1.1.359,5.6.2.1,2.7.4.24,2.7.11.26,2.7.4.21,2.1.1.310,2.1.1.202,2.1.1.203,3.2.1.105,2.1.1.167,2.3.1.296,2.1.1.221,3.6.5.5,2.7.11.1,2.4.1.136,3.1.13.1,3.-.-.-,3.1.13.1,True
3,P39286,3.6.1.-,3.6.1.-,100.0,True,False,1.0,3.6.1.-,1.0,0.940199,2.9.1.3,2.7.4.33,6.3.4.5,2.8.2.20,2.7.1.113,2.7.1.76,2.3.1.239,2.7.1.22,2.3.1.240,2.5.1.27,2.8.2.24,2.7.4.28,2.7.11.32,2.7.7.1,2.7.4.27,2.8.2.2,6.2.1.46,2.4.1.135,2.8.2.3,2.8.2.23,NaN,3.6.1.-,3.6.1.-,True
4,P56100,7.1.1.7,1.10.3.10,100.0,True,False,1.0,7.1.1.7,0.0,0.088895,7.1.1.2,7.1.1.7,2.1.1.295,3.1.3.78,7.1.1.9,2.1.1.86,4.2.1.78,2.5.1.21,4.4.1.20,7.2.4.2,1.17.4.4,4.3.2.9,1.11.1.11,1.8.3.2,3.1.26.3,2.4.1.132,2.4.1.257,1.1.99.22,3.3.2.11,3.2.2.5,1.10.3.10,NaN,7.1.1.7,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4433,P36837,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,3.2.1.81,4.1.2.9,3.2.1.31,1.17.2.3,1.21.3.3,1.17.9.1,1.3.3.14,1.1.3.45,2.4.1.352,4.1.2.22,2.7.8.23,1.13.11.15,2.4.1.232,4.2.1.140,5.5.1.20,3.2.1.75,1.1.3.42,2.7.1.185,1.13.11.49,1.7.2.8,NaN,1.-.-.-,3.2.1.81,False
4434,P77304,NaN,NaN,100.0,False,False,1.0,-,NaN,NaN,2.3.1.275,4.4.1.20,1.14.19.77,1.1.1.418,5.3.3.5,3.2.2.21,2.7.8.37,1.17.4.4,1.14.14.18,2.1.1.259,5.1.99.7,6.3.1.20,1.14.11.32,3.6.1.29,1.14.20.6,5.4.99.43,4.5.1.1,2.7.3.3,1.14.16.1,3.1.4.43,NaN,non Enzyme,2.3.1.275,False
4435,P27248,2.1.2.10,2.1.2.10,100.0,True,False,1.0,2.1.2.10,NaN,NaN,3.4.23.49,3.4.23.48,3.4.21.72,2.3.1.251,3.1.1.77,3.1.1.4,3.1.1.1,3.1.1.32,2.4.2.3,3.2.1.94,3.4.11.24,3.4.22.15,3.4.22.43,3.2.1.177,3.4.21.100,3.2.1.64,3.2.1.11,4.2.2.8,2.4.1.7,4.2.2.1,2.1.2.10,2.1.2.10,2.1.2.10,True
4436,P0AC88,4.2.1.47,4.2.1.47,100.0,True,False,1.0,4.2.1.47,NaN,NaN,4.1.3.27,3.5.1.42,2.4.2.4,1.1.1.338,2.7.7.6,3.6.1.64,3.6.1.22,2.7.1.134,3.6.1.55,2.7.1.159,2.4.1.62,4.2.99.21,5.4.99.43,3.5.1.90,6.3.2.33,4.2.1.31,3.6.1.62,5.4.4.2,5.4.99.23,3.6.1.45,4.2.1.47,4.2.1.-,4.2.1.47,True


In [258]:
res[(~res.ec_number.isnull()) & (res.ec_number_blast=='-')]

,id,ec_number,blattner_ec_number,pident,isenzyme_blast,isMultiFunctional_blast,functionCounts_blast,ec_number_blast,isenzyme_slice_pred,isenzyme_slice_prob,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,ec_number_deepec,ecpred_ec_number,ecpp,isenzyme_pred
126,P76555,2.7.2.1,NaN,100.0,False,False,1.0,-,1.0,0.896631,4.1.1.65,3.5.3.26,5.3.1.8,4.1.1.11,2.4.2.15,4.2.1.108,2.4.2.4,2.4.2.2,4.1.99.20,2.4.2.3,1.13.11.53,4.4.1.3,2.7.1.23,2.3.1.156,1.13.11.24,3.4.11.19,3.7.1.2,4.2.1.119,2.4.1.230,4.2.1.141,NaN,non Enzyme,4.1.1.65,False
264,P10408,7.4.2.8,NaN,100.0,False,False,1.0,-,1.0,0.890009,2.1.1.152,2.3.1.241,4.99.1.9,5.4.99.26,2.4.1.308,2.4.1.40,2.4.1.88,4.2.1.83,1.3.99.36,2.4.1.309,2.4.1.37,2.3.2.22,2.8.2.37,3.1.3.46,2.1.1.112,2.3.1.71,2.1.1.285,3.5.1.112,4.2.1.65,2.4.1.221,NaN,non Enzyme,2.1.1.152,False
1993,P0CF87,1.20.4.1,NaN,100.0,False,False,1.0,-,1.0,0.806698,2.1.1.57,6.1.1.16,1.8.5.1,3.4.21.91,4.2.3.47,6.1.1.10,3.1.4.4,5.4.4.7,4.2.3.24,1.14.19.3,4.2.3.65,4.2.3.67,4.2.3.113,4.2.3.88,2.1.1.275,3.4.22.46,5.5.1.17,1.20.4.2,2.7.7.30,4.2.3.77,1.20.4.1,1.-.-.-,2.1.1.57,False
2191,P77445,1.2.1.10,NaN,100.0,False,False,1.0,-,1.0,0.869030,2.7.1.71,6.3.3.3,1.2.1.41,1.1.1.302,3.2.2.9,4.2.1.113,1.2.7.10,7.6.2.8,1.14.14.155,6.3.5.5,5.3.99.11,3.7.1.12,2.1.1.289,4.1.1.95,2.5.1.105,2.7.1.40,3.1.4.55,3.1.3.32,2.7.1.185,1.14.13.20,NaN,1.2.1.-,2.7.1.71,False
2331,P76556,2.7.2.1,NaN,100.0,False,False,1.0,-,1.0,0.974506,2.7.7.1,2.4.2.21,2.1.1.274,2.7.4.2,2.1.1.285,4.2.1.167,2.1.1.273,4.2.1.175,1.21.99.4,2.4.2.9,4.1.3.32,3.1.3.32,2.7.4.33,4.2.1.54,2.4.1.309,2.1.1.258,3.4.23.51,3.1.3.102,2.7.1.206,1.2.1.58,NaN,3.-.-.-,2.7.7.1,False
2602,P77218,2.3.1.8,NaN,100.0,False,False,1.0,-,1.0,0.942506,2.7.1.71,6.3.3.3,2.1.1.199,7.1.2.2,2.7.13.3,2.7.4.23,4.1.1.48,6.3.2.8,2.7.4.8,2.7.4.7,2.1.1.242,2.1.1.104,6.3.3.2,7.1.1.8,2.5.1.3,2.1.1.35,2.7.4.25,3.1.3.68,2.7.4.3,3.1.3.22,NaN,2.-.-.-,2.7.1.71,False
3407,P45759,7.4.2.8,NaN,100.0,False,False,1.0,-,1.0,0.723751,3.6.4.12,7.4.2.10,7.2.2.5,7.4.2.9,7.3.2.6,7.2.2.7,7.6.2.8,7.5.2.1,7.2.2.16,7.2.2.20,2.7.1.21,7.6.2.4,7.2.2.11,2.7.7.49,7.6.2.10,7.5.2.12,2.7.1.138,7.3.2.2,7.2.2.4,7.6.2.14,NaN,3.6.-.-,3.6.4.12,False


In [265]:
sprot1515

id ec_number                                                seq  \
0     P42641   3.6.5.-  MKFVDEASILVVAGDGGNGCVSFRREKYIPKGGPDGGDGGDGGDVW...   
1     P0AE22   3.1.3.2  MRKITQAISAVCLLFALNSSAVALASSPSPLNPGTNVARLAEQAPI...   
2     P21499  3.1.13.1  MSQDPFQEREAEKYANPIPSREFILEHLTKREKPASRDELAVELHI...   
3     P39286   3.6.1.-  MSKNKLSKGQQRRVNANHQRRLKTSKEKPDYDDNLFGEPDEGIVIS...   
4     P56100   7.1.1.7              MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI   
...      ...       ...                                                ...   
4745  P36837       NaN  MNTTTPMGMLQQPRPFFMIFFVELWERFGYYGVQGVLAVFFVKQLG...   
4746  P77304       NaN  MSTANQKPTESVSLNAFKQPKAFYLIFSIELWERFGYYGLQGIMAV...   
4747  P27248  2.1.2.10  MAQQTPLYEQHTLCGARMVDFHGWMMPLHYGSQIDEHHAVRTDAGM...   
4748  P0AC88  4.2.1.47  MSKVALITGVTGQDGSYLAEFLLEKGYEVHGIKRRASSFNTERVDH...   
4749  P0A6N8       NaN  MPRANEIKKGMVLNYNGKLLLVKDIDIQSPTARGAATLYKMRFSDV...   

      isenzyme  isMultiFunctional blattner_id blattner_ec_number        f1  \
0         True              False       b3183                NaN  0.003056   
1         True              False       b4055            3.1.3.2  0.001026   
2         True              False       b4179            3.1.-.-  0.073632   
3         True              False       b4161            3.6.1.-  0.000775   
4         True              False       b4515          1.10.3.10  0.001811   
...        ...                ...         ...                ...       ...   
4745     False              False       b3496                NaN  0.000070   
4746     False              False       b1634                NaN  0.008628   
4747      True              False       b2905           2.1.2.10  0.006723   
4748      True              False       b2053           4.2.1.47  0.000665   
4749     False              False       b2171                NaN  0.000589   

            f2        f3        f4        f5        f6        f7        f8  \
0    -0.087184  0.043387 -0.018850 -0.623890  0.034710  0.133006 -0.056650   
1     0.618393  0.409527 -0.002331 -0.670928  0.036821 -0.148798 -0.038413   
2    -0.003258  0.008865 -0.012720 -0.072697 -0.026430 -0.033917 -0.000300   
3    -0.008305  0.041130 -0.001737 -0.156966  0.009515  0.400612 -0.021405   
4    -0.050181  0.037575 -0.005655  0.025414  0.003502 -0.096007 -0.006303   
...        ...       ...       ...       ...       ...       ...       ...   
4745 -0.021424  0.203301  0.025054  0.425925 -0.013487 -0.131236 -0.000778   
4746 -0.065382  0.048911 -0.215166 -0.197027  0.001949  0.045984 -0.001380   
4747  0.227349  0.029725  0.032317  0.064065  0.092823 -0.044260 -0.024524   
4748 -0.029179  0.024183 -0.005963 -0.175254  0.042627  0.554610 -0.020871   
4749  0.032014  0.013608 -0.008443 -0.115612  0.028364 -0.153695 -0.032118   

            f9       f10       f11       f12       f13       f14       f15  \
0    -0.004179 -0.002791 -0.002462  0.002599  0.048714  0.542259  0.009463   
1    -0.002616  0.026154  0.002405  0.001509  0.159823 -0.003070  0.004941   
2    -0.001022  0.041117 -0.046263  0.002500  0.003450 -0.647001  0.000169   
3    -0.002408  0.028499  0.002506  0.000343  0.094061  0.207324  0.066717   
4    -0.000473  0.012226 -0.007325  0.000229  0.079678  0.055803  0.013916   
...        ...       ...       ...       ...       ...       ...       ...   
4745 -0.000883 -0.099993  0.836958  0.000932  0.031655 -0.030638  0.086026   
4746 -0.004231  0.022663  0.821562 -0.000184  0.113237 -0.018413  0.045726   
4747 -0.001700  0.055556 -0.014932 -0.004102  0.006636 -0.052259  0.001718   
4748 -0.008380  0.105102  0.008972 -0.000733  0.003622  0.059358 -0.007454   
4749 -0.000680  0.051763 -0.121068 -0.000767  0.050263  0.110978  0.005803   

           f16       f17       f18       f19       f20       f21       f22  \
0    -0.000082 -0.009477 -0.090929  0.357896 -0.001747 -0.006085 -0.301930   
1     0.001484  0.026647 -0.172789  0.213775 -0.003496 -0.020142 -0.319490   
2     0.000002  0.045165  0.002784 -0.96